## MLFlow workflow example
> Example showing setting up a sample MLFLow project and tracking parameters, metrics and artifacts.

In [14]:
import os
import pathlib

import mlflow
from mlflow.data.filesystem_dataset_source import FileSystemDatasetSource

### Setup

In [15]:
mlflow.set_tracking_uri('https://mlflow.tenzing.shkhr.ovh')
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'https://minio-api.tenzing.shkhr.ovh'
EXPERIMENT_NAME = 'workflow-example'
ARTIFACT_LOCATION = f's3://mlflow/{EXPERIMENT_NAME}'

EXPERIMENT_DIR = pathlib.Path('/data/kaggle') / EXPERIMENT_NAME
EXPERIMENT_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# If the experiment does not exist, create the experiment for the first time and
# set the artifact location.
if mlflow.get_experiment_by_name(EXPERIMENT_NAME) is None:
    mlflow.create_experiment(EXPERIMENT_NAME, artifact_location=ARTIFACT_LOCATION)

experiment = mlflow.set_experiment(EXPERIMENT_NAME)

In [16]:
with mlflow.start_run(run_name='test', description='Test run for approach xyz.', experiment_id=experiment.experiment_id) as run:
    params = {
        'first_param': 42,
        'list_param': [1, 2, 3],
    }
    mlflow.log_params(params)
    mlflow.set_tags({
        'test_run': True,
        'missing_metrics': True
    })

    # Run the training/eval code.
    ...
    mlflow.log_metric('accuracy', 0.9)
    mlflow.log_metric('f1', 0.84)

    RUN_DIR = EXPERIMENT_DIR / run.info.run_id
    RUN_DIR.mkdir(parents=True, exist_ok=False)
    MODEL_PATH = RUN_DIR / 'model.pkl'

    with open(MODEL_PATH, 'w') as f:
        f.write('something')
    
    mlflow.log_artifact(MODEL_PATH)